In [ ]:
%pip install dexter-cqa==1.0.9
%pip install transformers==4.33.1

# Task 1: Evaluate LLMs on retrieved context documents

## 1.1 Extract contexts for each query using an off the shelf retriever (contriever)

In [ ]:
from dexter.retriever.dense.Contriever import Contriever
from dexter.config.constants import Split
from dexter.data.loaders.RetrieverDataset import RetrieverDataset
from dexter.utils.metrics.retrieval.RetrievalMetrics import RetrievalMetrics
from dexter.utils.metrics.SimilarityMatch import CosineSimilarity as CosScore
from dexter.data.datastructures.hyperparameters.dpr import DenseHyperParams
import json


config_instance = DenseHyperParams(query_encoder_path="facebook/contriever",
                                 document_encoder_path="facebook/contriever"
                                 ,batch_size=32)
config = config_instance.get_all_params()

loader = RetrieverDataset("wikimultihopqa","wikimultihopqa-corpus","/kaggle/input/2wikimultihop/config.ini",Split.DEV)
queries, qrels, corpus = loader.qrels()

# print(type(corpus))
# print("queries",len(queries),len(qrels),len(corpus),queries[0])
# tasb_search = Contriever(config_instance)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Transforming passage dataset: 100%|██████████| 563424/563424 [00:01<00:00, 295864.09it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Harley-Davidson Harley-Davidson
KeysView(<Section: Data-Path>)
12576


100%|██████████| 1200/1200 [03:05<00:00,  6.46it/s]


Dataset loaded of length 12000


In [ ]:
from collections import defaultdict
hard_negatives = defaultdict(dict)
positives = defaultdict(dict)
# filter docs to extract hard negatives
for query_id, relevant in docs['retrieved'].items():
  positive = qrels[query_id]
  for r, score in relevant.items():
      if r not in positive:
          hard_negatives[query_id][r] = score
      else:
          positives[query_id][r] = score

In [ ]:
relevant = json.load(open('/kaggle/input/2wikimultihop/retrieved_contexts.json'))
hard_negatives = json.load(open('/kaggle/input/2wikimultihop/hard_negatives.json'))
hard_positives = json.load(open('/kaggle/input/2wikimultihop/positives.json'))

## 1.2 Feed the retrieval results to the LLM and evaluate the accuracy using Exact Match (and metrics like ROUGE and BLEU)

### Few-Shot Prompts Evaluation

In [ ]:
import json
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from sklearn.metrics import f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import torch

# Load JSON files
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Map document IDs to their text
def get_document_texts(doc_ids, document_data):
    return [document_data[str(doc_id)]["text"] for doc_id in doc_ids if str(doc_id) in document_data]

def get_mix(topk, negatives, ratio, N, query_id):
    return topk[:N]

# Retrieve top-k contexts
def retrieve_contexts(query_id, contriever_data, hard_negs, document_data, ratio, N):
    doc_ids = list(contriever_data.get(query_id, {}).keys())
    hard_negs_id = list(hard_negs.get(query_id, {}).keys())
    doc_ids = get_mix(doc_ids, hard_negs_id, ratio, N, query_id)
    return get_document_texts(doc_ids, document_data)


# Prepare input for LLM
def prepare_input(query, contexts, zero_shot=False):
    if zero_shot:
        user_prompt = f"Given the evidence, Evidence: {' '.join(contexts)} \n use the information, think step by step and output the final answer extremely concisely in the form [Final Answer]: for the question, Question:{query}"
        return user_prompt
    user_prompt = f"""[Question]: When does monsoon season end in the state the area code 575 is located?
[Answer]: The area code 575 is located in New Mexico. Monsoon season in New Mexico typically ends in mid-September. So the
[Final Answer]: mid-September.
[Question]: What is the current official currency in the country where Ineabelle Diaz is a citizen?
[Answer]: Ineabelle Diaz is from Peurto Rico, which is in the United States of America. The current official currency in the United
States is the United States dollar.
[Final Answer]: United States dollar.
[Question]: Where was the person who founded the American Institute of Public Opinion in 1935 born?
[Answer]: The person who founded the American Institute of Public Opinion in 1935 is George Gallup. George Gallup was born
in Jefferson, Iowa.
[Final Answer]: Jefferson.
[Question]: What language is used by the director of Tiffany Memorandum?
[Answer]: The director of Tiffany Memorandum is Sergio Grieco. Sergio Grieco speaks Italian.
[Final Answer]: Italian.
[Question]: What is the sports team the person played for who scored the first touchdown in Superbowl 1?
[Answer]: The player that scored the first touchdown in Superbowl 1 is Max McGee. Max McGee played for the Green Bay
Packers.
[Final Answer]: Green Bay Packers.
[Question]: The birth country of Jayantha Ketagoda left the British Empire when?
[Answer]: The birth country of Jayantha Ketagoda is Sri Lanka. Sri Lanka left the British Empire on February 4, 1948. So the
[Final Answer]: February 4, 1948.\n\n Follow the above example and Given the evidence, Evidence: {' '.join(contexts)} \n use the information, think step by step and output the final answer extremely concisely in the form [Final Answer]: for the question, Question:{query}"""
    return user_prompt

# Generate response using LLM
def generate_answer(input_text, tokenizer, model):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to('cuda')
    inputs.pop("token_type_ids", None)
    outputs = model.generate(**inputs, max_new_tokens=1500)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# F1 Score Calculation
def f1_score_metric(prediction, ground_truth):
    pred_tokens = set(prediction.lower().split())
    gt_tokens = set(ground_truth.lower().split())

    intersection = pred_tokens.intersection(gt_tokens)
    precision = len(intersection) / len(pred_tokens) if len(pred_tokens) > 0 else 0
    recall = len(intersection) / len(gt_tokens) if len(gt_tokens) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1, precision, recall

# BLEU Score Calculation
def bleu_score_metric(prediction, ground_truth):
    prediction_tokens = prediction.lower().split()
    ground_truth_tokens = ground_truth.lower().split()
    score = sentence_bleu([ground_truth_tokens], prediction_tokens, smoothing_function=SmoothingFunction().method0)

    return score

# ROUGE Score Calculation
def rouge_score_metric(prediction, ground_truth):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth, prediction)
    return scores

# Process dataset and evaluate responses
def process_and_evaluate(contriever_data, dataset, hard_negs, document_data, tokenizer, model, ratio, N, zero_shot=False):
    results = {}
    question_df = {"questions":[],"answers":[]}
    total_em = 0
    total_f1 = 0
    total_precision = 0
    total_recall = 0
    total_bleu = 0
    total_rouge = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    count = 0
    failed = 0
    print(f"Processing with ratio {ratio}, N={N}")
    for entry in dataset:
        if count % 100 == 0:
          print(f"Processed {100*count/1200}% of queries")
        if count == 1200:
            break
        else:
            count += 1
        query_id = entry["_id"]
        query = entry["question"]
        ground_truth = entry["answer"]
        # Retrieve contexts and prepare input
        contexts = retrieve_contexts(query_id, contriever_data, hard_negs, document_data, ratio, N)
        # print(contexts)
        input_text = prepare_input(query, contexts, zero_shot)
        # print(input_text)

        # Generate response
        try:
            prediction = generate_answer(input_text, tokenizer, model)
        except RuntimeError as e:
            if failed == 0:
                print("Fails start at:", count)
            failed += 1
            continue


        # Calculate Exact Match score
        if "not possible" in prediction.lower() or "unknown" in prediction.lower():
            em = 0
        elif len(re.split(r'\[?Final Answer\]?:', prediction)) > 1:
            answer = re.split(r'\[?Final Answer\]?:', prediction)[-1]
            em = 1 if ground_truth.lower() in prediction.lower() else 0
        else:
            em = 0

        total_em += em

        # Calculate F1 Score
        f1,precision, recall = f1_score_metric(re.split(r'\[?Final Answer\]?:', prediction)[-1], ground_truth)
        total_f1 += f1
        total_precision += precision
        total_recall += recall

        # Calculate BLEU Score
        bleu = bleu_score_metric(re.split(r'\[?Final Answer\]?:', prediction)[-1], ground_truth)
        total_bleu += bleu

        # Calculate ROUGE Score
        rouge = rouge_score_metric(re.split(r'\[?Final Answer\]?:', prediction)[-1], ground_truth)
        total_rouge['rouge1'] += rouge['rouge1'].fmeasure
        total_rouge['rouge2'] += rouge['rouge2'].fmeasure
        total_rouge['rougeL'] += rouge['rougeL'].fmeasure

        # Store results
        results[query_id] = {"prediction": prediction, "ground_truth": ground_truth, "exact_match": em}

        if len(re.split(r'\[?Final Answer\]?:', prediction)) > 1:
            question_df["answers"].append(re.split(r'\[?Final Answer\]?:', prediction)[-1])
        else:
            question_df["answers"].append(prediction)

        question_df["questions"].append(query)

    # Calculate overall metrics
    accuracy = total_em / 1200
    average_f1 = total_f1 / 1200
    average_precision = total_precision / 1200
    average_recall = total_recall / 1200
    average_bleu = total_bleu / 1200
    average_rouge1 = total_rouge['rouge1'] / 1200
    average_rouge2 = total_rouge['rouge2'] / 1200
    average_rougeL = total_rouge['rougeL'] / 1200

    final_questions = pd.DataFrame(question_df)

    print("N Failed:", failed)
    print(f"Exact Match Accuracy: {accuracy * 100:.2f}%")
    print(f"Average F1 Score: {average_f1:.2f}")
    print(f"Average Precision: {average_precision:.2f}")
    print(f"Average Recall: {average_recall:.2f}")
    print(f"Average BLEU Score: {average_bleu:.2f}")
    print(f"Average ROUGE-1 F-Score: {average_rouge1:.2f}")
    print(f"Average ROUGE-2 F-Score: {average_rouge2:.2f}")
    print(f"Average ROUGE-L F-Score: {average_rougeL:.2f}")

    final_questions.to_csv("llama2_wqa_rag_5_few_shot_metrics.tsv", sep="\t", index=False)

    return results, accuracy, average_f1, average_precision, average_recall, average_bleu, average_rouge1, average_rouge2, average_rougeL

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

auth_token = os.getenv("huggingface_token")

if not auth_token:
    raise ValueError("Authentication token not found. Please set HF_TOKEN in your .env file.")

login(auth_token)

# File paths
dataset_file_path = "/kaggle/input/2wikimultihop/dev.json"  # Replace with actual path
document_file_path = "/kaggle/input/2wikimultihop/wiki_musique_corpus.json"  # Replace with actual path

# Load data
contriever_data = relevant
dataset = load_json(dataset_file_path)
document_data = load_json(document_file_path)

model_name = "meta-llama/Llama-2-7b-chat-hf"

# Initialize LLM and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

ratios = [1]
N = 5
ZERO_SHOT = False # Change this if you want to do zero shot/few shot

for ratio in ratios:
    # Process dataset and evaluate
    results, accuracy, average_f1, average_precision, average_recall, average_bleu, average_rouge1, average_rouge2, average_rougeL = process_and_evaluate(
        contriever_data, dataset, hard_negatives, document_data, tokenizer, model, ratio, N, zero_shot=ZERO_SHOT)

    print(f"Results for ratio {ratio}, N={N}")
    # Print evaluation results
    print(f"Overall Exact Match Accuracy: {accuracy * 100:.2f}%")
    print(f"Average F1 Score: {average_f1:.2f}")
    print(f"Average Precision: {average_precision:.2f}")
    print(f"Average Recall: {average_recall:.2f}")
    print(f"Average BLEU Score: {average_bleu:.2f}")
    print(f"Average ROUGE-1 F-Score: {average_rouge1:.2f}")
    print(f"Average ROUGE-2 F-Score: {average_rouge2:.2f}")
    print(f"Average ROUGE-L F-Score: {average_rougeL:.2f}")

    with open(f"/kaggle/working/accuracy_metrics_{ratio}_{N}_{ZERO_SHOT}.txt", "w") as f:
        f.write(f"Overall Exact Match Accuracy: {accuracy * 100:.2f}%\n")
        f.write(f"Average F1 Score: {average_f1:.2f}\n")
        f.write(f"Average Precision: {average_precision:.2f}\n")
        f.write(f"Average Recall: {average_recall:.2f}\n")
        f.write(f"Average BLEU Score: {average_bleu:.2f}\n")
        f.write(f"Average ROUGE-1 F-Score: {average_rouge1:.2f}\n")
        f.write(f"Average ROUGE-2 F-Score: {average_rouge2:.2f}\n")
        f.write(f"Average ROUGE-L F-Score: {average_rougeL:.2f}\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]